In [ ]:
import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
sys.path.append(os.path.abspath("D:\\burtm\\Visual_studio_code\\PD_related_projects"))

from utils.model_utils import get_model, get_trainable_layers
from utils.data_loading import get_dataloaders
from utils.utils_transforms import get_transform  
from utils.training_utils import fine_tune_last_n_layers, train_model

In [ ]:
selected_model = "resnet18"
selected_transform = "resnet18"
N_max=282
use_patches=True
pretrained=True
depth=2

In [ ]:
transform=get_transform(selected_transform,use_patches=use_patches)
train_dataloader,val_dataloader=get_dataloaders(transform, batch_size=16, N_max=282, file_name='train_df_patches_cc.csv')


In [ ]:
# Modify the final classification layer (assuming you have 10 classes)
num_classes = 2  # Change this to match your dataset

model=get_model(selected_model, pretrained=pretrained, num_classes=num_classes)

# Define loss function and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is: ",device)
model = model.to(device)
criterion = nn.CrossEntropyLoss()

In [ ]:
def progressive_unfreezing(model,model_name, train_loader,val_loader, 
                           criterion, device, base_lr=1e-3, num_epochs_per_stage=2, checkpoint_path=None):
    all_train_losses, all_val_losses = [], []
    # Freeze all layers except the classifier
    for param in model.parameters():
        param.requires_grad = False
    if model_name=='resnet18':
        for param in model.fc.parameters(): #for resnet the last concolutional layer is called fc
            param.requires_grad = True
        # Define optimizer for the classifier only
        optimizer = optim.Adam(model.fc.parameters(), lr=base_lr)
        layer_groups = [model.layer4, model.layer3, model.layer2, model.layer1]  # Deepest to shallowest layers
    else:
        print('model not supported')
        return
    
    print('Step 1: training classification head')
    model, train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, 
                                                  optimizer, device, num_epochs=num_epochs_per_stage,
                                                  checkpoint_path=checkpoint_path+'\classifier.pth')
    all_train_losses.extend(train_losses)
    all_val_losses.extend(val_losses)
    print('model on:',next(model.parameters()).device)

    # **Step 2: Unfreeze layers progressively**
    print('Step 2: Unfreeze layers progressively')
    lr = base_lr * 0.1  # Reduce learning rate for deeper layers

    for i,layer in enumerate(layer_groups):
        # Unfreeze the current layer
        for param in layer.parameters():
            param.requires_grad = True  

        # Define new optimizer with unfrozen layers
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

        # Train again with the newly unfrozen layer
        print(f"\nUnfreezing {layer} and training...")
        model, train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, 
                                                      optimizer, device, num_epochs=num_epochs_per_stage,
                                                      checkpoint_path=checkpoint_path+f'fine_tuning_layer_{i}.pth')
        all_train_losses.extend(train_losses)
        all_val_losses.extend(val_losses)
        print('model on:',next(model.parameters()).device)
        # Decrease learning rate for stability
        lr *= 0.1  
    print('fine tuning complete')
    return model, all_train_losses, all_val_losses

In [ ]:
model, train_losses,val_losses = progressive_unfreezing(model,selected_model, train_dataloader,val_dataloader, 
                                                        criterion, device, base_lr=1e-3, num_epochs_per_stage=2, 
                                                        checkpoint_path='D:\\burtm\\Visual_studio_code\\PD_related_projects\\checkpoints')